In [ ]:
# Package Imports - feel free to add what you think might be useful! 
%pip install requests
import requests

## Task 3: Quest for Quantum Randomness

### Part A: Truly Different

In [ ]:
""" 
Using the QRNGaaS User Manual provided, 
write Python code to get random numbers from IDQuantique's Quantum Random Number Generators. 

The API documentation is available at: https://drive.google.com/file/d/1OkyRUP7HC4dbmi8OvoT4EuvXqRVGdsjD/view
You'll probably reuse this code for Task 4. So write it well!

If you are unfamiliar with writing HTTP requests in Python, 
https://www.w3schools.com/python/module_requests.asp

If you're really stuck, this can definitely help. Though we encourage you to read the documentation yourself! 
https://curlconverter.com/python/
"""

In [ ]:
"""
To get started, we provide a simple example of how to use the API to generate 10 int16 random numbers.
In particular, observe where we pass in the min and max parameters to specify the range of random numbers we want
and the quantity parameter to specify how many random numbers we want.
Here is the API KEY you will need to use: aTo4BKRvnc49uRWDk034zaua87vGRXKk9TMLdfkI
"""

!curl "https://qrng.idqloud.com/api/1.0/short?max=32767&min=-32768&quantity=10" -X GET -H "X-API-KEY: <ADD API KEY FROM ABOVE HERE>"

In [ ]:
"""
You can paste the other environmental entropy based TRNGs from Task 2 and compare the performance
Compare your written QRNG to the previous TRNGs. Be sure to compare them with respect to bitrate and entropy.
"""
def plot_entropy():
    pass # TODO

def get_bitrate():
    pass # TODO

### Part B: Easy as Pi!

In [ ]:
"""
Implement the Monte Carlo algorithm for estimating Pi using your various sources of randomness. 

If you are stuck on how to implement the algorithm: 
https://www.geeksforgeeks.org/estimating-value-pi-using-monte-carlo/
"""
def estimate_pi():
    pass # TODO